In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

data = pd.read_csv("~/adult.csv")
print(data.head())
dummies = True
if(dummies):
    data = pd.get_dummies(data)
    data = data.drop(["income_<=50K"], axis=1)
    feature_names = list(data.columns[0:-1])
    label_name = data.columns[-1]
else:
    data["over"] = data["income"] == "<=50K"
    data = data.drop(["income"], axis=1)
    cat_columns = data.select_dtypes(['object']).columns
    print(cat_columns)
    data[cat_columns] = data[cat_columns].apply(lambda x: pd.factorize( x )[0])
    feature_names = list(data.columns[0:-1])
    label_name = data.columns[-1]
print(data.head())

   age workclass  fnlwgt  ... hours.per.week  native.country income
0   90         ?   77053  ...             40   United-States  <=50K
1   82   Private  132870  ...             18   United-States  <=50K
2   66         ?  186061  ...             40   United-States  <=50K
3   54   Private  140359  ...             40   United-States  <=50K
4   41   Private  264663  ...             40   United-States  <=50K

[5 rows x 15 columns]
   age  fnlwgt  ...  native.country_Yugoslavia  income_>50K
0   90   77053  ...                          0            0
1   82  132870  ...                          0            0
2   66  186061  ...                          0            0
3   54  140359  ...                          0            0
4   41  264663  ...                          0            0

[5 rows x 109 columns]


In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

data = pd.read_csv("~/higgs.csv")
print(data.head())
dummies = True
if(dummies):
    data = pd.get_dummies(data)
    feature_names = list(data.columns[0:-1])
    label_name = data.columns[-1]
else:
    data["over"] = data["income"] == "<=50K"
    data = data.drop(["income"], axis=1)
    cat_columns = data.select_dtypes(['object']).columns
    print(cat_columns)
    data[cat_columns] = data[cat_columns].apply(lambda x: pd.factorize( x )[0])
    feature_names = list(data.columns[0:-1])
    label_name = data.columns[-1]
print(data.head())

   lepton_pT REAL  lepton_eta REAL  ...  m_wwbb REAL  target
0        0.730938        -0.862016  ...     0.807032       0
1        0.824456        -0.017586  ...     1.101483       0
2        0.592217        -0.614628  ...     1.440432       0
3        0.474177        -0.863964  ...     0.863015       0
4        1.349509         1.182344  ...     0.778491       0

[5 rows x 25 columns]
   lepton_pT REAL  lepton_eta REAL  ...  m_wwbb REAL  target
0        0.730938        -0.862016  ...     0.807032       0
1        0.824456        -0.017586  ...     1.101483       0
2        0.592217        -0.614628  ...     1.440432       0
3        0.474177        -0.863964  ...     0.863015       0
4        1.349509         1.182344  ...     0.778491       0

[5 rows x 25 columns]


In [3]:
X = data[feature_names]
y = data[label_name]

X_mean = X.mean()
X_std = X.std()
eps = 1e-4
X = (X - X_mean) / (X_std + eps)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=123, shuffle=True
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, random_state=123, shuffle=True
)
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(18315, 108)
(6105, 108)
(8141, 108)


In [4]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

clf = XGBClassifier()
clf.fit(X_train, y_train)
y_probs = clf.predict_proba(X_test)[:, 1]
y_preds = clf.predict(X_test)
y_test = y_test.astype(int)
y_preds = y_test.astype(int)
roc_score = roc_auc_score(y_test, y_probs)
print(roc_score)
print(np.flip(np.argsort(clf.feature_importances_)))

0.923190260478814
[ 33   3   2  46   4  42  56  51  15  48  58  43  29   0   7  12   8  35
   5 105  64  11  54  27  96  50  37  68  59  71  60  39  61  63  52  49
  13  41  44  55 101  66   1  30  38  23  45  32  31  92  24  34  10   6
  53  26  99 102  47  77  17  57  78  18  19  90  14  72  21 104  16  67
  85  70  22  88  76  36  75 106  89 107  91  62 103  65 100  69  98  97
  95   9  94  93  86  28  74  40  79  80  81  20  82  83  84  87  25  73]


In [5]:
xgbclf = XGBClassifier(max_depth = 10)
xgbclf.fit(X_train, y_train)
y_probs = xgbclf.predict_proba(X_test)[:, 1]
y_preds = xgbclf.predict(X_test)
y_test = y_test.astype(int)
y_preds = y_test.astype(int)
roc_score = roc_auc_score(y_test, y_probs)
print(roc_score)

df = xgbclf._Booster.trees_to_dataframe()
df

0.9154998440074463


,Tree,Node,ID,Feature,Split,Yes,No,Missing,Gain,Cover
0,0,0,0-0,f33,0.080368,0-1,0-2,0-1,2640.145510,4578.750000
1,0,1,0-1,f3,0.811864,0-3,0-4,0-3,593.699219,2475.250000
2,0,2,0-2,f2,0.940338,0-5,0-6,0-5,1024.396240,2103.500000
3,0,3,0-3,f2,1.717695,0-7,0-8,0-7,87.056641,2430.250000
4,0,4,0-4,f0,-1.252271,0-9,0-10,0-9,7.664246,45.000000
...,...,...,...,...,...,...,...,...,...,...
15671,99,172,99-172,Leaf,NaN,NaN,NaN,NaN,0.149939,1.287332
15672,99,173,99-173,Leaf,NaN,NaN,NaN,NaN,-0.002828,68.208908
15673,99,174,99-174,Leaf,NaN,NaN,NaN,NaN,-0.145402,14.524666
15674,99,175,99-175,Leaf,NaN,NaN,NaN,NaN,0.132150,10.204220


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from tqdm import tqdm


def normalize_data(X):
    X_pd = pd.DataFrame(X)
    X_mean = np.array(X_pd.mean())
    X_std = np.array(X_pd.std())
    eps = 1e-4
    normalized_X = (X - X_mean) / (X_std + eps)
    return normalized_X, X_mean, X_std, eps

num_trees=1
num_samples = -1
path_to_train_data = {}
for row, label in tqdm(zip(X_train[:num_samples], y_train[:num_samples])):
    path = []
    for i in range(num_trees):
        starting_node = f"{i}-0"
        cur_node = df.loc[df["ID"] == starting_node]
        while (cur_node["Feature"] != "Leaf").item():
            feature_id = int(cur_node["Feature"].item().split("f")[1])
            feature_value = row[feature_id]
            if feature_value < cur_node["Split"].item():
                next_id = cur_node["Yes"].item()
            else:
                next_id = cur_node["No"].item()
            cur_node = df.loc[df["ID"] == next_id]
        last_node_id = cur_node["ID"].item()
        path.append(last_node_id)
    path = tuple(path)

    data, labels = path_to_train_data.get(path, ([], []))
    data.append(row)
    labels.append(label)
    path_to_train_data[path] = (data, labels)

18314it [06:12, 49.21it/s]


In [7]:
for k, v in path_to_train_data.items():
    data, labels = v
    print(k, len(labels))

('0-258',) 53
('0-73',) 4654
('0-221',) 2206
('0-277',) 281
('0-77',) 824
('0-43',) 248
('0-236',) 26
('0-222',) 193
('0-229',) 366
('0-301',) 15
('0-223',) 128
('0-266',) 234
('0-297',) 1220
('0-71',) 198
('0-292',) 193
('0-275',) 224
('0-274',) 812
('0-269',) 90
('0-245',) 444
('0-270',) 7
('0-175',) 8
('0-273',) 1067
('0-225',) 511
('0-27',) 376
('0-257',) 423
('0-247',) 90
('0-127',) 4
('0-261',) 340
('0-37',) 4
('0-299',) 19
('0-97',) 35
('0-291',) 61
('0-298',) 60
('0-253',) 637
('0-289',) 20
('0-20',) 166
('0-217',) 4
('0-206',) 10
('0-148',) 9
('0-279',) 33
('0-98',) 19
('0-230',) 60
('0-129',) 294
('0-31',) 26
('0-171',) 143
('0-262',) 55
('0-302',) 6
('0-296',) 34
('0-227',) 6
('0-238',) 15
('0-101',) 95
('0-83',) 19
('0-60',) 11
('0-239',) 126
('0-281',) 7
('0-263',) 26
('0-255',) 21
('0-286',) 17
('0-201',) 6
('0-74',) 6
('0-85',) 35
('0-159',) 10
('0-246',) 13
('0-219',) 7
('0-282',) 9
('0-145',) 21
('0-233',) 72
('0-260',) 62
('0-90',) 9
('0-104',) 13
('0-118',) 7
('0-231

In [8]:
path_to_model = {}
for k in path_to_train_data.keys():
    data, labels = path_to_train_data.get(k, ([], []))
    data = np.array(data)
    labels = np.array(labels)
    if len(np.unique(labels)) == 2:
        lr_clf = LogisticRegression()
        data, data_mean, data_std, eps = normalize_data(data)
        lr_clf.fit(data, labels)
        path_to_model[k] = lr_clf, data_mean, data_std, eps
        print(len(labels))

53


4654
2206
281
824
26


193
366
15
128
234
1220
198
193


224
812
90
444
7
1067
511
423


90
4
340
4
19
35
61
60
637
20
4
10
9


33
19
60
294
26
143
55
6
34
6
15
95
126
26


21
17
6
6
35
13
7
9
72
62
9
13
7
33
25
31
4
4


6
7
48
35
7
4
4
7
36
7
17
5
63
4
11
21
9
6
6
4
4
5
11
7
6


5
4
7
13
8
4
5
7
11
9
5
6
7
5
9
9
11
7
4
5
7
4
4
4
4


5
4
11
4
5
5
7
4
4
4
5
5
4


In [9]:
path_to_val_data = {}
for row, label in tqdm(zip(X_val[:num_samples], y_val[:num_samples])):
    path = []
    for i in range(num_trees):
        starting_node = f"{i}-0"
        cur_node = df.loc[df["ID"] == starting_node]
        while (cur_node["Feature"] != "Leaf").item():
            feature_id = int(cur_node["Feature"].item().split("f")[1])
            feature_value = row[feature_id]
            if feature_value < cur_node["Split"].item():
                next_id = cur_node["Yes"].item()
            else:
                next_id = cur_node["No"].item()
            cur_node = df.loc[df["ID"] == next_id]
        last_node_id = cur_node["ID"].item()
        path.append(last_node_id)
    path = tuple(path)

    data, labels = path_to_val_data.get(path, ([], []))
    data.append(row)
    labels.append(label)
    path_to_val_data[path] = (data, labels)

6104it [02:02, 49.95it/s]


In [10]:
for k in path_to_val_data.keys():
    data, labels = path_to_val_data[k]
    data = np.array(data)
    labels = np.array(labels)
    v = path_to_model.get(k, None)
    if(v is not None and len(np.unique(labels))==2):
        lr_clf, data_mean, data_std, eps = v
        xgb_rocauc = roc_auc_score(labels, clf.predict_proba(data)[:, 1])
        xgb_acc = accuracy_score(labels, clf.predict(data))
        data = (data - data_mean) / (data_std + eps)
        lr_rocauc = roc_auc_score(labels, lr_clf.predict_proba(data)[:, 1])
        lr_acc = accuracy_score(labels, lr_clf.predict(data))
        print(lr_rocauc-xgb_rocauc, np.sum(labels==0), np.sum(labels==1), len(labels), k)
        print(lr_acc-xgb_acc, len(labels), k)
        print()

-0.015593220338983027 120 295 415 ('0-297',)
0.0 415 ('0-297',)

-0.02314814814814814 1512 14 1526 ('0-73',)
-0.000655307994757437 1526 ('0-73',)

-0.2602739726027397 73 6 79 ('0-266',)
-0.025316455696202556 79 ('0-266',)

-0.12930232558139532 43 50 93 ('0-277',)
-0.05376344086021512 93 ('0-277',)

0.09361447584866156 743 9 752 ('0-221',)
-0.0013297872340425343 752 ('0-221',)

-0.07657541322314065 128 121 249 ('0-274',)
-0.04016064257028107 249 ('0-274',)

-0.047704330836860875 249 111 360 ('0-273',)
-0.019444444444444375 360 ('0-273',)

-0.10561056105610578 101 30 131 ('0-257',)
-0.022900763358778664 131 ('0-257',)

0.011018463371054277 146 23 169 ('0-225',)
-0.023668639053254448 169 ('0-225',)

-0.0185185185185186 3 36 39 ('0-101',)
0.0 39 ('0-101',)

-0.3090909090909091 11 5 16 ('0-258',)
-0.3125 16 ('0-258',)

-0.02777777777777768 108 2 110 ('0-129',)
0.0 110 ('0-129',)

-0.3012820512820512 2 78 80 ('0-71',)
-0.012500000000000067 80 ('0-71',)

-0.11274509803921573 170 30 200 ('0-25

-0.1515151515151516 22 3 25 ('0-171',)
-0.07999999999999996 25 ('0-171',)

0.015625 64 1 65 ('0-222',)
0.0 65 ('0-222',)

-0.40909090909090917 10 11 21 ('0-260',)
-0.19047619047619052 21 ('0-260',)

-0.25 1 4 5 ('0-299',)
0.0 5 ('0-299',)

-0.11428571428571421 10 7 17 ('0-269',)
0.11764705882352944 17 ('0-269',)

-1.0 1 10 11 ('0-296',)
-0.09090909090909094 11 ('0-296',)

-0.055555555555555525 9 2 11 ('0-263',)
-0.09090909090909094 11 ('0-263',)

-0.2727272727272727 11 1 12 ('0-33',)
-0.16666666666666663 12 ('0-33',)

-0.5277777777777777 9 8 17 ('0-236',)
-0.35294117647058826 17 ('0-236',)

0.0 2 1 3 ('0-259',)
-0.6666666666666666 3 ('0-259',)



-1.1102230246251565e-16 7 5 12 ('0-241',)
0.08333333333333326 12 ('0-241',)

-0.06481481481481483 9 12 21 ('0-262',)
0.09523809523809529 21 ('0-262',)

-0.5 3 4 7 ('0-201',)
-0.4285714285714286 7 ('0-201',)

0.625 8 1 9 ('0-232',)
-0.2222222222222222 9 ('0-232',)

-1.0 2 1 3 ('0-125',)
-0.6666666666666667 3 ('0-125',)

-0.38888888888888895 9 2 11 ('0-231',)
-0.27272727272727276 11 ('0-231',)

-0.5 4 1 5 ('0-120',)
-0.4 5 ('0-120',)

0.0 1 2 3 ('0-217',)
0.3333333333333333 3 ('0-217',)

-0.4 35 3 38 ('0-223',)
-0.02631578947368418 38 ('0-223',)

-0.33333333333333326 3 1 4 ('0-301',)
0.0 4 ('0-301',)

-0.6666666666666667 4 3 7 ('0-204',)
-0.14285714285714285 7 ('0-204',)

-0.16842105263157892 19 5 24 ('0-284',)
-0.16666666666666674 24 ('0-284',)

0.0 4 3 7 ('0-248',)
-0.14285714285714285 7 ('0-248',)

-0.6666666666666666 3 1 4 ('0-282',)
-0.75 4 ('0-282',)

1.0 1 3 4 ('0-148',)
0.25 4 ('0-148',)

0.0 5 1 6 ('0-182',)
-0.16666666666666674 6 ('0-182',)

-0.33766233766233766 7 11 18 ('0-291

0.0 1 10 11 ('0-97',)
-0.09090909090909094 11 ('0-97',)

-1.0 2 2 4 ('0-300',)
-0.5 4 ('0-300',)

-0.25 4 4 8 ('0-286',)
-0.25 8 ('0-286',)

0.5 2 1 3 ('0-246',)
0.0 3 ('0-246',)

-0.38461538461538464 9 13 22 ('0-298',)
-0.3181818181818181 22 ('0-298',)

-0.03296703296703296 13 7 20 ('0-267',)
0.15000000000000002 20 ('0-267',)

-0.5 4 1 5 ('0-289',)
0.0 5 ('0-289',)

0.16666666666666663 4 3 7 ('0-235',)
-0.1428571428571429 7 ('0-235',)

-0.3333333333333333 2 3 5 ('0-294',)
-0.19999999999999996 5 ('0-294',)

0.0 1 1 2 ('0-226',)
0.0 2 ('0-226',)

0.0 1 2 3 ('0-31',)
-0.33333333333333337 3 ('0-31',)

-0.11111111111111116 3 3 6 ('0-206',)
0.16666666666666669 6 ('0-206',)

0.0 1 1 2 ('0-271',)
0.5 2 ('0-271',)

-0.14285714285714296 7 5 12 ('0-85',)
-0.08333333333333331 12 ('0-85',)

0.0 1 1 2 ('0-224',)
-0.5 2 ('0-224',)

-0.20000000000000007 5 1 6 ('0-251',)
0.0 6 ('0-251',)

-0.20512820512820512 13 3 16 ('0-233',)
-0.0625 16 ('0-233',)

0.4375 4 4 8 ('0-279',)
0.0 8 ('0-279',)

-1.0 1 1 

In [11]:
path_to_test_data = {}
for row, label in tqdm(zip(X_test[:num_samples], y_test[:num_samples])):
    path = []
    for i in range(num_trees):
        starting_node = f"{i}-0"
        cur_node = df.loc[df["ID"] == starting_node]
        while (cur_node["Feature"] != "Leaf").item():
            feature_id = int(cur_node["Feature"].item().split("f")[1])
            feature_value = row[feature_id]
            if feature_value < cur_node["Split"].item():
                next_id = cur_node["Yes"].item()
            else:
                next_id = cur_node["No"].item()
            cur_node = df.loc[df["ID"] == next_id]
        last_node_id = cur_node["ID"].item()
        path.append(last_node_id)
    path = tuple(path)

    data, labels = path_to_test_data.get(path, ([], []))
    data.append(row)
    labels.append(label)
    path_to_test_data[path] = (data, labels)

8140it [02:40, 50.72it/s]


In [12]:
for k in path_to_test_data.keys():
    data, labels = path_to_test_data[k]
    data = np.array(data)
    labels = np.array(labels)
    v = path_to_model.get(k, None)
    if(v is not None and len(np.unique(labels))==2):
        lr_clf, data_mean, data_std, eps = v
        xgb_rocauc = roc_auc_score(labels, clf.predict_proba(data)[:, 1])
        xgb_acc = accuracy_score(labels, clf.predict(data))
        data = (data - data_mean) / (data_std + eps)
        lr_rocauc = roc_auc_score(labels, lr_clf.predict_proba(data)[:, 1])
        lr_acc = accuracy_score(labels, lr_clf.predict(data))
        print(lr_rocauc-xgb_rocauc, np.sum(labels==0), np.sum(labels==1), len(labels), k)
        print(lr_acc-xgb_acc, len(labels), k)
        print()

-0.07738095238095233 72 28 100 ('0-275',)
-0.05999999999999994 100 ('0-275',)

-0.05088459302758863 169 397 566 ('0-297',)
0.017667844522968212 566 ('0-297',)

-0.13871333657429918 2057 15 2072 ('0-73',)
-0.0009652509652509078 2072 ('0-73',)

-0.16676700782661047 151 22 173 ('0-229',)
-0.028901734104046284 173 ('0-229',)

-0.15910823170731703 256 41 297 ('0-253',)
-0.010101010101010166 297 ('0-253',)

-0.17500000000000004 8 5 13 ('0-240',)
-0.07692307692307698 13 ('0-240',)

-0.004233934789490301 312 162 474 ('0-273',)
0.016877637130801704 474 ('0-273',)

0.005573192239858971 175 162 337 ('0-274',)
-0.005934718100890191 337 ('0-274',)

-0.10946859903381645 115 45 160 ('0-261',)
-0.018750000000000044 160 ('0-261',)

-0.09738884968242767 218 39 257 ('0-225',)
-0.015564202334630406 257 ('0-225',)

-0.0757593167107129 967 27 994 ('0-221',)
-0.005030181086519203 994 ('0-221',)

-0.7336956521739131 2 92 94 ('0-71',)
-0.021276595744680882 94 ('0-71',)

-0.2407407407407407 6 9 15 ('0-85',)
-0.

-0.5 2 1 3 ('0-204',)
-0.6666666666666667 3 ('0-204',)

-0.25 2 2 4 ('0-234',)
0.0 4 ('0-234',)

1.0 1 2 3 ('0-127',)
0.0 3 ('0-127',)

-0.02941176470588236 17 12 29 ('0-291',)
-0.1724137931034483 29 ('0-291',)

-0.11728395061728403 18 9 27 ('0-267',)
-0.07407407407407407 27 ('0-267',)

-0.1428571428571428 7 1 8 ('0-187',)
0.0 8 ('0-187',)

0.07738095238095244 12 14 26 ('0-262',)
0.1923076923076923 26 ('0-262',)

0.008888888888888946 45 5 50 ('0-223',)
-0.16000000000000003 50 ('0-223',)

-0.18181818181818177 1 11 12 ('0-97',)
0.08333333333333326 12 ('0-97',)

-0.25 20 1 21 ('0-33',)
-0.04761904761904756 21 ('0-33',)

0.0 1 4 5 ('0-81',)
-0.19999999999999996 5 ('0-81',)

-0.20000000000000007 5 4 9 ('0-236',)
-0.11111111111111105 9 ('0-236',)

-0.2719298245614036 19 6 25 ('0-258',)
-0.24 25 ('0-258',)

-0.6666666666666667 1 3 4 ('0-214',)
-0.25 4 ('0-214',)

0.0 4 1 5 ('0-116',)
-0.39999999999999997 5 ('0-116',)

0.0 2 5 7 ('0-285',)
0.28571428571428575 7 ('0-285',)

0.0 1 7 8 ('0-238',)

-0.25 1 4 5 ('0-293',)
0.0 5 ('0-293',)

0.16666666666666663 3 4 7 ('0-294',)
0.0 7 ('0-294',)

0.0 1 3 4 ('0-300',)
-0.5 4 ('0-300',)

0.0 2 1 3 ('0-37',)
0.0 3 ('0-37',)

-0.6666666666666666 2 3 5 ('0-206',)
-0.19999999999999996 5 ('0-206',)

-0.18749999999999994 16 3 19 ('0-241',)
0.0 19 ('0-241',)

-0.5 2 2 4 ('0-118',)
-0.25 4 ('0-118',)

0.0 1 1 2 ('0-288',)
-0.5 2 ('0-288',)

-0.8333333333333333 3 2 5 ('0-166',)
-0.6000000000000001 5 ('0-166',)

0.0 2 1 3 ('0-154',)
0.0 3 ('0-154',)

-1.0 1 2 3 ('0-286',)
0.0 3 ('0-286',)

0.0 1 1 2 ('0-227',)
0.0 2 ('0-227',)

-0.5 1 2 3 ('0-237',)
0.0 3 ('0-237',)



In [28]:
lr_training_data_amount = 0
total_data_amount = 0
for k in path_to_train_data.keys():
    data, labels = path_to_train_data[k]
    data = np.array(data)
    labels = np.array(labels)
    total_data_amount += len(labels)
    v = path_to_model.get(k, None)
    if(v is not None and len(np.unique(labels))==2):
        lr_clf, data_mean, data_std, eps = v
        xgb_rocauc = roc_auc_score(labels, clf.predict_proba(data)[:, 1])
        xgb_acc = accuracy_score(labels, clf.predict(data))
        data = (data - data_mean) / (data_std + eps)
        lr_rocauc = roc_auc_score(labels, lr_clf.predict_proba(data)[:, 1])
        lr_acc = accuracy_score(labels, lr_clf.predict(data))
        print(lr_acc-xgb_acc, len(labels), k)
        #if(xgb_rocauc>lr_rocauc or False):
        if(k!="0-67"):
            path_to_model[k] = None
        else:
            lr_training_data_amount += len(labels)
print(lr_training_data_amount/total_data_amount)

-0.0013053519429661042 9959 0-49
-0.033399800598205354 2006 0-67
-0.014285714285714235 70 0-62


-0.0981625975333501 3973 0-68
-0.005331753554502439 1688 0-51


-0.06461675579322634 2244 0-78
-0.0023809523809523725 420 0-63


-0.029055690072639195 826 0-64
-0.0019193857965451588 521 0-27
-0.062146892655367214 177 0-75
-0.008379888268156499 358 0-57


-0.008021390374331583 374 0-71
0.04878048780487798 41 0-58
-0.16393442622950816 61 0-76
0.038461538461538436 26 0-54
-0.01379310344827589 290 0-47


-0.043243243243243246 555 0-52
0.0 14 0-65
0.00561797752808979 178 0-69
-0.09333333333333338 75 0-70
0.2222222222222222 9 0-50
0.0 8 0-74
-0.01526717557251911 131 0-61
0.0 50 0-60


0.0 15 0-55
0.04255319148936165 47 0-77
0.0714285714285714 14 0-53
0.0 4 0-73
0.0 5 0-24
0.2222222222222222 9 0-59


0.0 5 0-37
0.0 4 0-72
0.0 4 0-79
0.0 6 0-56
0.0 4 0-28


0.0 4 0-9
0.08214914615668127


In [29]:
path_to_model

{'0-49': None,
 '0-67': (LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                     intercept_scaling=1, l1_ratio=None, max_iter=100,
                     multi_class='auto', n_jobs=None, penalty='l2',
                     random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                     warm_start=False),
  array([-6.55477963e-01,  3.84737096e-02, -1.78420638e-01, -1.20763829e-01,
         -1.26236094e-01,  2.30143759e-01, -1.25530706e-01, -1.00559010e-01,
         -6.69357080e-02, -1.45642445e-02,  1.72177550e-01, -6.23116994e-02,
         -2.86934731e-03, -2.28352986e-02,  3.29033539e-03, -1.71647725e-01,
         -1.93379968e-01, -1.15988910e-01, -7.19145286e-02, -1.01547054e-01,
         -1.42167670e-01, -1.26541484e-01,  9.30905944e-02,  1.87483898e-01,
         -4.43530594e-01, -1.13240941e-01,  5.02256528e-01, -2.36264742e-01,
         -3.95069151e-02, -1.34091753e-01,  2.13784487e-01, -3.97386196e-01,
         -2.64868476e-02

In [31]:
y_probs = []
y_preds = []
num_lr = 0
data, labels = path_to_test_data["0-67"]
data = np.array(data)
labels = np.array(labels)
print(data.shape)
print(labels.shape)
for row, label in tqdm(zip(data, labels)):
    cur_node = df.loc[df["ID"] == starting_node]
    while((cur_node["Feature"] != "Leaf").item()):
        feature_id = int(cur_node["Feature"].item().split("f")[1])
        feature_value = row[feature_id]
        if(feature_value<cur_node["Split"].item()):
            next_id = cur_node["Yes"].item()
        else:
            next_id = cur_node["No"].item()
        cur_node = df.loc[df["ID"] == next_id]
    last_node_id = cur_node["ID"].item()

    row = row.reshape(1, -1)
    v = path_to_model.get(last_node_id, None)
    ###
    #if(last_node_id != "0-68"):
    #    v = None
    ###
    if(v is not None):
        lr_clf, data_mean, data_std, eps = v
        row = (row - data_mean) / (data_std + eps)
        y_prob = lr_clf.predict_proba(row)[:, 1]
        y_pred = lr_clf.predict(row)
        y_probs.append(y_prob)
        y_preds.append(y_pred)
        num_lr += 1
    else:
        continue
        y_prob = clf.predict_proba(row)[:, 1]
        y_pred = clf.predict(row)
        y_probs.append(y_prob)
        y_preds.append(y_pred)

print("XGBoost + LR Leaf Nodes ROCAUC:", roc_auc_score(labels, y_probs))
print("XGBoost ROCAUC:", roc_auc_score(labels, clf.predict_proba(data)[:, 1]))
print("XGBoost + LR Leaf Nodes accuracy:", accuracy_score(labels, y_preds))
print("XGBoost accuracy:", accuracy_score(labels, clf.predict(data)))
print("Fraction of test set using LR leaf nodes:", num_lr/len(y_probs))

(698, 108)
(698,)


698it [00:05, 125.36it/s]


XGBoost + LR Leaf Nodes ROCAUC: 0.7027958686928492
XGBoost ROCAUC: 0.7291165054930596
XGBoost + LR Leaf Nodes accuracy: 0.8051575931232091
XGBoost accuracy: 0.8008595988538681
Fraction of test set using LR leaf nodes: 1.0


In [ ]:
y_probs = []
y_preds = []
num_lr = 0
for row, label in tqdm(zip(X_test[:num_samples], y_test[:num_samples])):
    cur_node = df.loc[df["ID"] == starting_node]
    while((cur_node["Feature"] != "Leaf").item()):
        feature_id = int(cur_node["Feature"].item().split("f")[1])
        feature_value = row[feature_id]
        if(feature_value<cur_node["Split"].item()):
            next_id = cur_node["Yes"].item()
        else:
            next_id = cur_node["No"].item()
        cur_node = df.loc[df["ID"] == next_id]
    last_node_id = cur_node["ID"].item()

    row = row.reshape(1, -1)
    v = path_to_model.get(last_node_id, None)
    ###
    #if(last_node_id != "0-68"):
    #    v = None
    ###
    if(v is not None):
        lr_clf, data_mean, data_std, eps = v
        row = (row - data_mean) / (data_std + eps)
        y_prob = lr_clf.predict_proba(row)[:, 1]
        y_pred = lr_clf.predict(row)
        y_probs.append(y_prob)
        y_preds.append(y_pred)
        num_lr += 1
    else:
        continue
        y_prob = clf.predict_proba(row)[:, 1]
        y_pred = clf.predict(row)
        y_probs.append(y_prob)
        y_preds.append(y_pred)

print("XGBoost + LR Leaf Nodes ROCAUC:", roc_auc_score(y_test[:num_samples], y_probs))
print("XGBoost ROCAUC:", roc_auc_score(y_test[:num_samples], clf.predict_proba(X_test[:num_samples])[:, 1]))
print("XGBoost + LR Leaf Nodes accuracy:", accuracy_score(y_test[:num_samples], y_preds))
print("XGBoost accuracy:", accuracy_score(y_test[:num_samples], clf.predict(X_test[:num_samples])))
print("Fraction of test set using LR leaf nodes:", num_lr/len(y_probs))

In [ ]:
print(roc_auc_score(y_test[:num_samples], clf.predict_proba(X_test[:num_samples])[:, 1]))
print(accuracy_score(y_test[:num_samples], clf.predict(X_test[:num_samples])))

In [ ]:
test_row = X_test[0,:].reshape(1,-1)
clf.predict_proba(test_row)[:, 1].item()


In [74]:
dump_list=clf.get_booster().get_dump()
num_t=len(dump_list)

preds = [0]
for n in range(num_t):
  preds.append(clf.predict(X_test[0,:].reshape(1,-1), ntree_limit=n+1, output_margin=True))

print([preds[i+1]-preds[i] for i in range(num_t)])
print(sum([preds[i+1]-preds[i] for i in range(num_t)]))
print(clf.predict(X_test[:10,:], ntree_limit=0, output_margin=True))

[array([-0.14920795], dtype=float32), array([-0.17368156], dtype=float32), array([-0.10154185], dtype=float32), array([-0.08626291], dtype=float32), array([-0.04556143], dtype=float32), array([-0.09627342], dtype=float32), array([0.06078643], dtype=float32), array([0.00191265], dtype=float32), array([0.06866819], dtype=float32), array([-0.01632679], dtype=float32), array([-0.04130489], dtype=float32), array([-0.00127423], dtype=float32), array([0.00169301], dtype=float32), array([-0.02734345], dtype=float32), array([-0.04659027], dtype=float32), array([0.04050803], dtype=float32), array([0.00024194], dtype=float32), array([0.02009928], dtype=float32), array([0.01745957], dtype=float32), array([0.00433379], dtype=float32), array([0.0049423], dtype=float32), array([-0.00521648], dtype=float32), array([0.0203945], dtype=float32), array([-0.02368069], dtype=float32), array([0.00076413], dtype=float32), array([0.0346114], dtype=float32), array([-0.03332043], dtype=float32), array([0.0006735

In [75]:
print(dump_list[0])

0:[f33<0.0803676248] yes=1,no=2,missing=1
	1:[f3<0.81186378] yes=3,no=4,missing=3
		3:[f2<1.32901645] yes=7,no=8,missing=7
			7:[f4<5.28884888] yes=15,no=16,missing=15
				15:[f5<0.329016775] yes=29,no=30,missing=29
					29:[f58<2.10946441] yes=49,no=50,missing=49
						49:leaf=-0.578319788
						50:leaf=-0.0461538509
					30:[f2<0.940338016] yes=51,no=52,missing=51
						51:leaf=-0.519905508
						52:leaf=-0.350983918
				16:[f64<0.359547794] yes=31,no=32,missing=31
					31:[f0<-0.152607441] yes=53,no=54,missing=53
						53:leaf=-0.0666666701
						54:leaf=0.400000036
					32:[f2<0.74599874] yes=55,no=56,missing=55
						55:leaf=-0.347368419
						56:leaf=0.120000005
			8:[f5<0.248028904] yes=17,no=18,missing=17
				17:[f4<5.84597588] yes=33,no=34,missing=33
					33:[f2<2.10637355] yes=57,no=58,missing=57
						57:leaf=-0.444198906
						58:leaf=-0.120000005
					34:leaf=0.381818205
				18:[f0<-0.519162059] yes=35,no=36,missing=35
					35:[f35<0.366053462] yes=59,no=60,missing=59
				

In [ ]:
print(X_test.shape)
print(y_test.shape)

best_roc_score = 0
for i in tqdm(range(1, 11)):
    for j in range(1, 1000, 50):
        depth_clf = XGBClassifier(max_depth=i, n_estimators = j)
        depth_clf.fit(X_train, y_train)
        y_probs = depth_clf.predict_proba(X_test)[:, 1]
        y_preds = depth_clf.predict(X_test)
        roc_score = roc_auc_score(y_test, y_probs)
        acc_score = accuracy_score(y_test, y_preds)
        print(i, j)
        if(best_roc_score<roc_score):
            best_roc_score = roc_score
            print(roc_score)
            print(acc_score)
            print()

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

model = XGBClassifier()

n_estimators = range(1, 100, 5)


param_grid = dict(n_estimators=n_estimators)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)

grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_["mean_test_score"]
stds = grid_result.cv_results_["std_test_score"]
params = grid_result.cv_results_["params"]
print()
for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from tqdm import tqdm


def normalize_data(X):
    X_pd = pd.DataFrame(X)
    X_mean = np.array(X_pd.mean())
    X_std = np.array(X_pd.std())
    eps = 1e-4
    normalized_X = (X - X_mean) / (X_std + eps)
    return normalized_X, X_mean, X_std, eps


X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, random_state=123, shuffle=True)

for i in range(100):
    starting_node = f"{i}-0"
    print(starting_node)
    num_samples = -1
    path_to_train_data = {}
    for row, label in tqdm(zip(X_train2[:num_samples], y_train2[:num_samples])):
        cur_node = df.loc[df["ID"] == starting_node]
        while (cur_node["Feature"] != "Leaf").item():
            feature_id = int(cur_node["Feature"].item().split("f")[1])
            feature_value = row[feature_id]
            if feature_value < cur_node["Split"].item():
                next_id = cur_node["Yes"].item()
            else:
                next_id = cur_node["No"].item()
            cur_node = df.loc[df["ID"] == next_id]
        last_node_id = cur_node["ID"].item()

        data, labels = path_to_train_data.get(last_node_id, ([], []))
        data.append(row)
        labels.append(label)
        path_to_train_data[last_node_id] = (data, labels)

    path_to_val_data = {}
    for row, label in tqdm(zip(X_val[:num_samples], y_val[:num_samples])):
        cur_node = df.loc[df["ID"] == starting_node]
        while (cur_node["Feature"] != "Leaf").item():
            feature_id = int(cur_node["Feature"].item().split("f")[1])
            feature_value = row[feature_id]
            if feature_value < cur_node["Split"].item():
                next_id = cur_node["Yes"].item()
            else:
                next_id = cur_node["No"].item()
            cur_node = df.loc[df["ID"] == next_id]
        last_node_id = cur_node["ID"].item()

        data, labels = path_to_val_data.get(last_node_id, ([], []))
        data.append(row)
        labels.append(label)
        path_to_val_data[last_node_id] = (data, labels)

    # print("train")
    # for k in path_to_train_data.keys():
    #    data, labels = path_to_train_data[k]
    #    print(k, np.sum(np.array(labels) == 0), np.sum(np.array(labels) == 1), len(labels))
    #
    # print("test")
    # for k in path_to_test_data.keys():
    #    data, labels = path_to_test_data[k]
    #    print(k, np.sum(np.array(labels) == 0), np.sum(np.array(labels) == 1), len(labels))

    path_to_model = {}
    for k in path_to_train_data.keys():
        data, labels = path_to_test_data.get(k, ([], []))
        data = np.array(data)
        labels = np.array(labels)
        if len(np.unique(labels)) == 2:
            lr_clf = LogisticRegression()
            data, data_mean, data_std, eps = normalize_data(data)
            lr_clf.fit(data, labels)
            path_to_model[k] = lr_clf, data_mean, data_std, eps

    lr_training_data_amount = 0
    total_data_amount = 0
    for k in path_to_val_data.keys():
        data, labels = path_to_val_data[k]
        data = np.array(data)
        labels = np.array(labels)
        total_data_amount += len(labels)
        v = path_to_model.get(k, None)
        if v is not None and len(np.unique(labels)) == 2:
            lr_clf, data_mean, data_std, eps = v
            data = (data - data_mean) / (data_std + eps)
            lr_rocauc = roc_auc_score(labels, lr_clf.predict_proba(data)[:, 1])
            xgb_rocauc = roc_auc_score(labels, clf.predict_proba(data)[:, 1])
            if xgb_rocauc > lr_rocauc:
                path_to_model[k] = None
            else:
                lr_training_data_amount += len(labels)

    y_probs = []
    y_preds = []
    num_lr = 0
    for row, label in tqdm(zip(X_test[:num_samples], y_test[:num_samples])):
        cur_node = df.loc[df["ID"] == starting_node]
        while (cur_node["Feature"] != "Leaf").item():
            feature_id = int(cur_node["Feature"].item().split("f")[1])
            feature_value = row[feature_id]
            if feature_value < cur_node["Split"].item():
                next_id = cur_node["Yes"].item()
            else:
                next_id = cur_node["No"].item()
            cur_node = df.loc[df["ID"] == next_id]
        last_node_id = cur_node["ID"].item()

        row = row.reshape(1, -1)
        v = path_to_model.get(last_node_id, None)
        ###
        # if(last_node_id != "0-68"):
        #    v = None
        ###
        if v is not None:
            lr_clf, data_mean, data_std, eps = v
            row = (row - data_mean) / (data_std + eps)
            y_prob = lr_clf.predict_proba(row)[:, 1]
            y_pred = lr_clf.predict(row)
            y_probs.append(y_prob)
            y_preds.append(y_pred)
            num_lr += 1
        else:
            y_prob = clf.predict_proba(row)[:, 1]
            y_pred = clf.predict(row)
            y_probs.append(y_prob)
            y_preds.append(y_pred)

    print("XGBoost + LR Leaf Nodes ROCAUC:", roc_auc_score(y_test[:num_samples], y_probs))
    print("XGBoost ROCAUC:", roc_auc_score(y_test[:num_samples], clf.predict_proba(X_test[:num_samples])[:, 1]))
    print("XGBoost + LR Leaf Nodes accuracy:", accuracy_score(y_test[:num_samples], y_preds))
    print("XGBoost accuracy:", accuracy_score(y_test[:num_samples], clf.predict(X_test[:num_samples])))
    print("Fraction of test set using LR leaf nodes:", num_lr / len(y_probs))
    print()